# Домашнее задание 2. Аэропорты. File-1
### Панкратов Никита

#### Импортируем необходимые для работы библиотеки

In [1]:
import pandas as pd
import numpy as np

### Считываем данные для работы

In [2]:
airport_row = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', 
                          names=['Airport_ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude',
                                  'Timezone', 'DST', 'Tz_database_time_zone', 'Type', 'Source'])


rout_row = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', 
                       names=['Airline', 'Airline_ID', 'Source_airport', 'Source_airport_ID', 'Destination_airport',
                             'Destination_airport_ID', 'Codeshare', 'Stops', 'Equipment'])

### Смотрим на данные

In [3]:
airport_row.head()

,Airport_ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz_database_time_zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
rout_row.head()

,Airline,Airline_ID,Source_airport,Source_airport_ID,Destination_airport,Destination_airport_ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


### Заменяем '\N' на np.NaN, поскольку '\N' - означает пропущенное значение, согласно объяснению на сайте. + Удаляем столбец 'ICAO', так как в дальнейшем будем работать только с аэропортами, имеющими код 'IATA'.

In [5]:
airport_row.drop(columns='ICAO', inplace=True)
airport_row['IATA'] = airport_row['IATA'].replace('\\N', np.nan)

airport_row.head()
#airport_row.tail() #появились ли NaN вместо '\N'

,Airport_ID,Name,City,Country,IATA,Latitude,Longitude,Altitude,Timezone,DST,Tz_database_time_zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


### Так как мы будем работаь только с теми аэропртами, которые иеют три знака (IATA), удалим те аэропорты, которые не имеют аббревиатуру IATA

In [6]:
print(airport_row['IATA'].isna().sum())
airport_row.dropna(subset=['IATA'], inplace=True)
print(airport_row['IATA'].isna().sum())

1626
0


### Заменим также в таблице rout_row пропущенные значения в столбцах с ID-аэропортов прилета и вылета на np.NaN.
##### P.s. здесь просто удалять строки будет неверно, так как может быть совершен вылет и не быть прилета и наоборот.

In [7]:
rout_row.head()
rout_row['Source_airport_ID'] = rout_row['Source_airport_ID'].replace('\\N', np.nan)
rout_row['Destination_airport_ID'] = rout_row['Destination_airport_ID'].replace('\\N', np.nan)


### Поскольку столбец 'Airline' хранит именно 2-символьные обзозначения для IATA (это соотвествует определению имеющихся данных в столбцах на сайте, с которого были взяты данные!), удалим строки, которые содержат len(Airline) != 2

In [8]:
rout_row = rout_row[rout_row['Airline'].apply(lambda x:len(x) == 2)]
#rout_row = rout_row[rout_row['Source_airport'].apply(lambda x:len(x) == 3)]  #не обязательно, так как на предыдущим шаге, 
                                                                               #по сути мы уже оставили только IATA (но чтобы 
                                                                                #не было ошибок в дальнейшем, я решил лучше 
                                                                                 #почистить и для двух других колонок по 
                                                                                  #описанию их с сайта)
#rout_row = rout_row[rout_row['Destination_airport'].apply(lambda x:len(x) == 3)]
rout_row.head()

,Airline,Airline_ID,Source_airport,Source_airport_ID,Destination_airport,Destination_airport_ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


### Удалим ненужные для дальнейшей работы колонки

In [9]:
airport_row.drop(columns=['Type', 'Source'], inplace=True)  # для всех строк одинаковое зн-ие и нам не потребуется эти столбики в дальнейшем

### Считаем через метод value_counts количество прилетов и вылетов для каждого аэропорта

In [10]:
#Сначала мерджим по Destination_airport и считаем кол-во прилетов для каждого аэропорта
merge_table = pd.merge(airport_row, rout_row, left_on='IATA', right_on='Destination_airport', how='inner')
dest_count = merge_table.groupby(['Country', 'IATA', 'Name']).Destination_airport.value_counts().reset_index(level='Destination_airport', drop=True)

#Затем делаем все аналогично, но уже по Source_airport
merge_table = pd.merge(airport_row, rout_row, left_on='IATA', right_on='Source_airport', how='inner')
source_count = merge_table.groupby(['Country', 'IATA', 'Name']).Source_airport.value_counts().reset_index(level='Source_airport', drop=True)

#Объединяем полученные результаты в одну табличку, заполняя нулями те значение, которые равны=NaN (это появилось из-за того, 
 # что возможно из аэропорта, допустим, был совершен вылет, Но не было прилетов (или наоборот))
prepared_ans = pd.concat([source_count, dest_count], axis=1).fillna(value=0).astype(int)

### Теперь подсчитаем сумму количества вылетов и прилетов по каждому коду IATA и выведем ТОП-10 (по макс.кол-ву такой суммы).

In [11]:
prepared_ans['Total_departure_arrival'] = prepared_ans.Source_airport + prepared_ans.Destination_airport
answer_1 = prepared_ans[['Source_airport', 'Destination_airport', 'Total_departure_arrival']].nlargest(10, 'Total_departure_arrival').reset_index(level='IATA', drop=True)
answer_1  

Source_airport  \
Country        Name                                                               
United States  Hartsfield Jackson Atlanta International Airport             915   
               Chicago O'Hare International Airport                         558   
China          Beijing Capital International Airport                        535   
United Kingdom London Heathrow Airport                                      527   
France         Charles de Gaulle International Airport                      524   
Germany        Frankfurt am Main Airport                                    497   
United States  Los Angeles International Airport                            492   
               Dallas Fort Worth International Airport                      469   
               John F Kennedy International Airport                         456   
Netherlands    Amsterdam Airport Schiphol                                   453   

                                                                 Destination_airport  \
Country        Name                                                                    
United States  Hartsfield Jackson Atlanta International Airport                  911   
               Chicago O'Hare International Airport                              550   
China          Beijing Capital International Airport                             534   
United Kingdom London Heathrow Airport                                           524   
France         Charles de Gaulle International Airport                           516   
Germany        Frankfurt am Main Airport                                         493   
United States  Los Angeles International Airport                                 498   
               Dallas Fort Worth International Airport                           467   
               John F Kennedy International Airport                              455   
Netherlands    Amsterdam Airport Schiphol                                        450   

                                                                 Total_departure_arrival  
Country        Name                                                                       
United States  Hartsfield Jackson Atlanta International Airport                     1826  
               Chicago O'Hare International Airport                                 1108  
China          Beijing Capital International Airport                                1069  
United Kingdom London Heathrow Airport                                              1051  
France         Charles de Gaulle International Airport                              1040  
Germany        Frankfurt am Main Airport                                             990  
United States  Los Angeles International Airport                                     990  
               Dallas Fort Worth International Airport                               936  
               John F Kennedy International Airport                                  911  
Netherlands    Amsterdam Airport Schiphol                                            903

# Таким образом, получили топ-10 аэропортов, сгруппированных по странам и имеющих максимальное суммарное кол-во прилетов и вылетов.